<a href="https://colab.research.google.com/github/RohiniShankari/GenAIcohort_May2025/blob/main/GenAIcohort_May2025_RAG_Rohini_Sankari.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install langchain sentence-transformers faiss-cpu groq gradio langchain-groq


In [2]:
pip install -U langchain-community

In [15]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get("groq_key")



In [4]:
from google.colab import files


filepath="/content/The National Health Mission (NHM),.txt"

with open(filepath, 'r', encoding='utf-8') as f:
    raw_text = f.read()

print("Text Loaded:", raw_text)

Text Loaded: The National Health Mission (NHM), the country’s flagship health systems strengthening programme,
particularly for primary and secondary health care envisages “attainment of universal access to equitable,
affordable and quality health care which is accountable and responsive to the needs of people”. Investments
during the life of the NHM in its earlier phases were targeted to strengthen Reproductive and Child Health
(RCH) services and contain the increasing burden of communicable diseases such as Tuberculosis, HIV/
AIDS and vector borne diseases. While such a focus on selective primary health care interventions, enabled
improvements in key indicators related to RCH and select communicable diseases, the range of services
delivered at the primary care level did not consider increasing disease burden and rising costs of care on
account of chronic diseases.
Studies show that 11.5% households in rural areas and about only 4% in urban areas, reported seeking any
form of OPD care

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.create_documents([raw_text])


In [7]:
docs[1]

Document(metadata={}, page_content='(RCH) services and contain the increasing burden of communicable diseases such as Tuberculosis, HIV/\nAIDS and vector borne diseases. While such a focus on selective primary health care interventions, enabled\nimprovements in key indicators related to RCH and select communicable diseases, the range of services\ndelivered at the primary care level did not consider increasing disease burden and rising costs of care on\naccount of chronic diseases.')

In [11]:
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings

embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectordb = FAISS.from_documents(docs, embedding)


In [16]:
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq

llm = ChatGroq(
    groq_api_key=os.environ["GROQ_API_KEY"],
    model_name="llama3-8b-8192"
)

retriever = vectordb.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)


In [17]:
query = "What services are provided at the Health and Wellness Centres?"
answer = qa_chain.run(query)
print("Answer:", answer)


Answer: According to the text, the guidelines for operationalizing the Health and Wellness Centres (HWCs) are intended to provide an overview of the systems requirements and strategies for change management to deliver Comprehensive Primary Health Care (CPHC) services. However, the specific services to be provided at the HWCs are not explicitly mentioned in the provided text.


In [18]:
import gradio as gr

def answer_query(query):
    return qa_chain.run(query)

gr.Interface(fn=answer_query,
             inputs="text",
             outputs="text",
             title="Ayushman Bharat Assistant (Groq + LLaMA3)",
             description="Ask questions about Ayushman Bharat from pages 19–21").launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://508614ef154f3b65bd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
